In [1]:
import os

from pydantic import BaseModel, Field
from typing import List, Annotated, Literal

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv

load_dotenv()

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
gemini_api_key=os.getenv("GEMINI_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [2]:
from agents import create_web_crawler_and_study_materials_agent

In [40]:
agent=create_web_crawler_and_study_materials_agent(model="gemini")

In [41]:
class Jobs(BaseModel):
    company: str = Field(description="Extract the name of the company that posted the job.",default="Not Specified")
    location: str = Field(description="Extract the location.",default="remote")
    job_role: str = Field(description="Extract the job role.",default="Not Specified")
    skills_required: list[str] = Field(description="Extract all the skills (technical and non-technical)required for the particular job.",default="Not specified")
    salary: str = Field(description="Extract the salary the company is offering for that particukar job.",default="Not Specified")

In [42]:
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate

parser=PydanticOutputParser(pydantic_object=Jobs)
str_parser=StrOutputParser()

text="""
    SOHAM MUKHERJEE
Kolkata, India | +91 8240771772 | soumyapu31@gmail.com | LinkedIn | Website | GitHub
SUMMARY
Aspiring Machine Learning & AI engineer with expertise in Deep Learning and Generative AI. Strong foundation in Neural Networks, Computer Vision, and NLP, with a focus on building intelligent and scalable AI solutions. Passionate about pushing the boundaries of AI-driven innovation. Also, a quick learner and a problem solver.
SKILLS
 Python
 Computer Vision
 Transformers
 RAG
 MySQL
 Object Detection
 LLMs
 AI Agents
 Machine Learning
 NLP
 Fine-tuning
 LangChain
 Neural Networks
 Embeddings
 Vector Databases
 HuggingFace
PROJECTS
Real-time Google Search AI Agent with Memory | GitHub Repository
• Developed a real-time AI-powered Google search agent using LangChain and Groq Inferencing Engine (Llama-3.1-8B model) to retrieve and summarize web data dynamically.
• Implemented conversational memory using LangChain's ConversationBufferMemory to retain context for follow-up queries, enhancing response relevance.
• Integrated Google Programmable Search API for structured web search results, formatted into markdown for improved readability in a Streamlit-based UI.
PDF Question Answering RAG | GitHub Repository
• Built a PDF-based Q&A system using Google's Gemini API to extract and answer questions from documents.
• Integrated LangChain for efficient document processing and retrieval-augmented generation (RAG).
• Developed a user-friendly Streamlit UI for seamless interaction and question-answering.
Facial Recognition Attendance System | GitHub Repository
• Used OpenCV’s Haar cascades to detect faces and face_recognition library to generate 128-dimensional embeddings for each face.
• Compared embeddings with a stored database using Euclidean distance to accurately identify students.
• Stores student data in Firebase Realtime Database and images in Firebase Storage for real-time attendance tracking.
Multi-purpose AI Assistant | GitHub Repository
• Multimodal AI Q&A System using Google’s Gemini API to handle text, image, and invoice-based queries.
• Streamlit Web Interface with dedicated pages for chatbot, text Q&A, vision-based queries, and invoice extraction..
• Gemini-Powered Features including a chatbot, invoice data extraction, vision-based answering, and text analysis in Markdown..
Automatic License Plate Detector | GitHub Repository
• License plate detection after fine-tuning a pre-trained YOLOv8 model on our custom dataset.
• Text extraction using Tesseract OCR to recognize the license plate numbers.
• Visualizing detected plates and extracted text in the web interface built using Streamlit.
Food Detector | GitHub Repository
• Initial classification with a CNN baseline model.
• Transfer learning with EfficientNet and ResNet, and enhanced via data augmentation.
• Fine-tuned pre-trained models to get better results.
CERTIFICATIONS
• Python for Data Science & Machine Learning [Udemy (URL)]
• TensorFlow Developer Certification [Zero to Mastery (URL)]
EDUCATION
Master of Science (M.Sc.) in Data Science & Analytics (2023-present)
NSHM College of Management & Technology
Bachelor of Science (B.Sc.) in Data Science (2020-2023)
iLEAD
"""

model=ChatGroq(
    model="llama-3.3-70b-specdec",
    api_key=groq_api_key
)

prompt1=PromptTemplate(
    template="""
        Summarize the entire profile of the candidate in question and extract their projects and skilsl from the resume I give below.\n
        {resume}.
    """,
    input_variables=["resume"]
)

summary_chain=prompt1|model|str_parser
summary=summary_chain.invoke({
    "resume":text
})



In [43]:
print(summary)

**Summary of the Candidate:**
Soham Mukherjee is an aspiring Machine Learning and AI engineer with expertise in Deep Learning and Generative AI. He has a strong foundation in Neural Networks, Computer Vision, and NLP, with a focus on building intelligent and scalable AI solutions. He is passionate about pushing the boundaries of AI-driven innovation and is a quick learner and problem solver. He is currently pursuing his Master's degree in Data Science & Analytics and has a Bachelor's degree in Data Science.

**Skills:**

1. Programming languages: Python
2. AI and ML: Machine Learning, Deep Learning, Neural Networks, NLP, Computer Vision
3. Tools and frameworks: Transformers, RAG, LangChain, HuggingFace, MySQL, Object Detection, LLMs, AI Agents
4. Technologies: OpenCV, face_recognition, Firebase Realtime Database, Firebase Storage, Streamlit, Tesseract OCR, YOLOv8, EfficientNet, ResNet
5. Databases: Vector Databases, MySQL

**Projects:**

1. **Real-time Google Search AI Agent with Memor

In [149]:
agent_prompt_template="""
    You are a highly skilled job search assistant named Friday. Your task is to analyze a candidate's resume and search for 5 currently open job postings that best match their profile.

    Use up-to-date sources (like online job listings or search engines) to find **real** and **ongoing** job opportunities that fit the candidate’s experience, skills, and qualifications.

    For each job posting, provide the following details in a structured format:

    1. Company Name  
    2. Job Role  
    3. Skills Required  
    4. Location  
    5. Salary (if mentioned; otherwise say "Not specified")
    6. Application Link (direct link to apply for the job)

    Be accurate and do not invent information. Only return actual job postings that are live on the internet right now. Focus on relevance and quality over quantity. Use the tools at your disposal.

    Here is the resume:
    {resume}
"""

# industry_trends=agent.invoke({"messages":agent_prompt_template.format(resume=summary)})

In [57]:
print(industry_trends["messages"][-1].content)

I'm sorry, but I was unable to find any relevant job postings using the tools available to me.


In [50]:
result=industry_trends["messages"][-1].content

prompt2=PromptTemplate(
    template="""
        Structure the following -> {skills}. \n{format_instructions}
    """,
    input_variables=["skills"],
    partial_variables={
        "format_instructions":parser.get_format_instructions()
    }
)

final_chain=prompt2|model|parser
final_result=final_chain.invoke({"skills":result})

OutputParserException: Failed to parse Jobs from completion [{"company": "Not Specified", "location": "remote", "job_role": "Machine Learning Engineer", "skills_required": ["Python", "Machine Learning", "Deep Learning", "Neural Networks", "NLP", "TensorFlow", "PyTorch"], "salary": "Not Specified"}, {"company": "Not Specified", "location": "remote", "job_role": "AI Engineer", "skills_required": ["Python", "Machine Learning", "Deep Learning", "NLP", "Computer Vision", "Generative AI models (GANs, VAEs, Transformers)"], "salary": "Not Specified"}, {"company": "Not Specified", "location": "remote", "job_role": "Computer Vision Engineer", "skills_required": ["Python", "OpenCV", "YOLOv8", "EfficientNet", "ResNet", "Machine Learning", "Deep Learning"], "salary": "Not Specified"}, {"company": "Not Specified", "location": "remote", "job_role": "NLP Engineer", "skills_required": ["Python", "Transformers", "RAG", "LangChain", "HuggingFace", "NLP techniques"], "salary": "Not Specified"}, {"company

In [51]:
final_result

NameError: name 'final_result' is not defined

In [150]:
from langgraph_agent import create_langgraph_agent

agent=create_langgraph_agent(model="gemini")

In [63]:
config={"configurable":{"thread_id":"1"}}
events = agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)

outputs=[]
# for event in events:
#     if "messages" in event:
#         event["messages"][-1].pretty_print()

for event in events:
    if "messages" in event:
        outputs.extend(event["messages"])

In [70]:
events = agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        outputs.extend(event["messages"])

In [72]:
events = agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Can you help me do it?"
                    "Can you give me a step by step walkthrough"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        outputs.extend(event["messages"])

In [84]:
# outputs[-1].content

print('\n'.join(outputs[-1].content))

I can provide a general step-by-step walkthrough to help you get started, but keep in mind that building an autonomous agent is a complex task, and the specific steps will depend on the agent's purpose and design.

**I. Project Setup and Initialization**

1.  **Install LangGraph and LangChain:**

    
```bash
    pip install langgraph langchain
    ```


2.  **Set up your environment:** Make sure you have Python installed (preferably 3.8 or higher). You'll also need an API key for a Large Language Model provider like OpenAI. Set the API key as an environment variable:

    
```bash
    export OPENAI_API_KEY="YOUR_API_KEY"
    ```


**II. Define the Agent's State**

1.  **Define the State Class:** The state represents the agent's memory and current understanding. It should include all the information the agent needs to make decisions. For example:

    
```python
    from typing import TypedDict, List, Dict
    from langchain.schema import BaseMessage

    class AgentState(TypedDict):
 

In [91]:
' '.join(outputs[-1].content)

'I can provide a general step-by-step walkthrough to help you get started, but keep in mind that building an autonomous agent is a complex task, and the specific steps will depend on the agent\'s purpose and design.\n\n**I. Project Setup and Initialization**\n\n1.  **Install LangGraph and LangChain:**\n\n     ```bash\n    pip install langgraph langchain\n    ``` \n\n2.  **Set up your environment:** Make sure you have Python installed (preferably 3.8 or higher). You\'ll also need an API key for a Large Language Model provider like OpenAI. Set the API key as an environment variable:\n\n     ```bash\n    export OPENAI_API_KEY="YOUR_API_KEY"\n    ``` \n\n**II. Define the Agent\'s State**\n\n1.  **Define the State Class:** The state represents the agent\'s memory and current understanding. It should include all the information the agent needs to make decisions. For example:\n\n     ```python\n    from typing import TypedDict, List, Dict\n    from langchain.schema import BaseMessage\n\n    c

In [151]:
from langchain_core.prompts import PromptTemplate

prompt=PromptTemplate(
    template="Convert the following into markdown -> {result}",
    input_variables=["result"],
)

markdown_chain=prompt|model|str_parser
result=markdown_chain.invoke({"result":' '.join(outputs[-1].content)})

In [152]:
from IPython.display import Markdown,display
display(Markdown(result))

### Introduction
I am still unable to reliably extract all the required information to fulfill the request completely. However, here are some potential leads based on the search results. Please note that the information might be incomplete, and it is recommended to verify the details on the linked pages.

### Job Leads
1. **Company Name:** Observe AI
    * **Job Role:** Machine Learning Engineer - NLP
    * **Skills Required:** Machine Learning, Deep Learning, NLP
    * **Location:** Not specified in the summary, check the link for details.
    * **Salary:** Not specified
    * **Application Link:** [https://www.moaijobs.com/job/machine-learning-engineer-nlp-observe-ai-4210](https://www.moaijobs.com/job/machine-learning-engineer-nlp-observe-ai-4210)
2. **Company Name:** IQVIA
    * **Job Role:** AI Engineer
    * **Skills Required:** Python, TensorFlow, PyTorch, Hugging Face (likely others, see link)
    * **Location:** Bengaluru
    * **Salary:** Not specified
    * **Application Link:** [https://www.linkedin.com/posts/niranjan-m-084a4145_iqvia-now-hiring-ai-engineer-activity-72324485556892444288-aVZT](https://www.linkedin.com/posts/niranjan-m-084a4145_iqvia-now-hiring-ai-engineer-activity-72324485556892444288-aVZT)
3. **Company Name:** Coditas
    * **Job Role:** Generative AI Engineer - Python
    * **Skills Required:** Azure, Python, TensorFlow, PyTorch, scikit-learn
    * **Location:** Not specified, check the link for details.
    * **Salary:** Not specified
    * **Application Link:** [https://www.linkedin.com/posts/coditas_hiring-now-langchain-rag-activity-7272508141416689664-BlDP](https://www.linkedin.com/posts/coditas_hiring-now-langchain-rag-activity-7272508141416689664-BlDP)

### Conclusion
I am continuing to improve my ability to extract this information. In the meantime, please use these leads as a starting point for your job search and verify the details on the application pages.

In [94]:
def get_response(user_input:str,config:dict=config):
    events = agent.stream(
        {
            "messages": [
                {
                    "role": "user",
                    "content": (
                        user_input
                    ),
                },
            ],
        },
        config,
        stream_mode="values",
    )
    for event in events:
        if "messages" in event:
            outputs.extend(event["messages"])

    prompt=PromptTemplate(
        template="Convert the following into markdown -> {result}",
        input_variables=["result"],
    )

    markdown_chain=prompt|model|str_parser
    result=markdown_chain.invoke({"result":' '.join(outputs[-1].content)})

    return result

In [95]:
result2=get_response("Give me an explanation of generative AI")

In [96]:
display(Markdown(result2))

### Generative AI
Generative AI refers to a class of artificial intelligence algorithms that learn from existing data (text, images, audio, etc.) to generate new, similar data. It's like teaching a computer to imitate an artist, writer, or composer, and then letting it create its own original works in that style.

#### Key Concepts
* **Learning from Data**: Generative AI models are trained on large datasets. They analyze the patterns, structures, and relationships within the data. For example, a model trained on images of cats learns what features (ears, fur, whiskers, etc.) are characteristic of cats.
* **Generating New Data**: Once trained, these models can generate new data points that resemble the training data but are not identical to it. The "newness" can range from slight variations on existing examples to completely original creations. In the cat example, the model could generate a picture of a cat it has never seen before.
* **Types of Generative Models**: There are several different architectures used for generative AI, including:
	+ **Generative Adversarial Networks (GANs)**: GANs involve two neural networks, a generator and a discriminator. The generator creates new data, and the discriminator tries to distinguish between the generated data and real data from the training set. The generator and discriminator are trained in competition, leading to increasingly realistic generated data.
	+ **Variational Autoencoders (VAEs)**: VAEs learn a compressed representation of the input data (a "latent space"). They can then sample from this latent space and decode the samples to generate new data points.
	+ **Transformers**: While originally designed for natural language processing, transformers have proven highly effective for generative tasks in other domains, such as image and audio generation. They use a mechanism called "self-attention" to weigh the importance of different parts of the input data when generating new data.
	+ **Autoregressive Models**: These models predict the next element in a sequence based on the preceding elements. They are commonly used for text and music generation.

#### Applications of Generative AI
Generative AI has a wide range of applications, including:
* **Image Generation**: Creating realistic images of people, objects, and scenes.
* **Text Generation**: Writing articles, poems, scripts, and code.
* **Music Generation**: Composing original music in various styles.
* **Video Generation**: Creating short videos or animations.
* **Drug Discovery**: Designing new molecules with desired properties.
* **Product Design**: Generating new product ideas and prototypes.
* **Data Augmentation**: Creating synthetic data to improve the performance of other machine learning models.

#### Limitations and Challenges
* **Data Dependency**: Generative AI models require large amounts of training data.
* **Bias**: Generated data can reflect biases present in the training data.
* **Lack of Control**: It can be difficult to control the specific characteristics of the generated data.
* **Ethical Concerns**: Concerns about misuse, such as creating deepfakes or generating misleading content.
* **Computational Cost**: Training generative models can be computationally expensive.

In summary, generative AI is a powerful technology that allows computers to create new data that resembles the data they were trained on. It has numerous applications across various industries, but it also presents ethical and practical challenges that need to be addressed.

In [97]:
result3=get_response("make it a bit shorter")

In [98]:
display(Markdown(result2))

### Generative AI
Generative AI refers to a class of artificial intelligence algorithms that learn from existing data (text, images, audio, etc.) to generate new, similar data. It's like teaching a computer to imitate an artist, writer, or composer, and then letting it create its own original works in that style.

#### Key Concepts
* **Learning from Data**: Generative AI models are trained on large datasets. They analyze the patterns, structures, and relationships within the data. For example, a model trained on images of cats learns what features (ears, fur, whiskers, etc.) are characteristic of cats.
* **Generating New Data**: Once trained, these models can generate new data points that resemble the training data but are not identical to it. The "newness" can range from slight variations on existing examples to completely original creations. In the cat example, the model could generate a picture of a cat it has never seen before.
* **Types of Generative Models**: There are several different architectures used for generative AI, including:
	+ **Generative Adversarial Networks (GANs)**: GANs involve two neural networks, a generator and a discriminator. The generator creates new data, and the discriminator tries to distinguish between the generated data and real data from the training set. The generator and discriminator are trained in competition, leading to increasingly realistic generated data.
	+ **Variational Autoencoders (VAEs)**: VAEs learn a compressed representation of the input data (a "latent space"). They can then sample from this latent space and decode the samples to generate new data points.
	+ **Transformers**: While originally designed for natural language processing, transformers have proven highly effective for generative tasks in other domains, such as image and audio generation. They use a mechanism called "self-attention" to weigh the importance of different parts of the input data when generating new data.
	+ **Autoregressive Models**: These models predict the next element in a sequence based on the preceding elements. They are commonly used for text and music generation.

#### Applications of Generative AI
Generative AI has a wide range of applications, including:
* **Image Generation**: Creating realistic images of people, objects, and scenes.
* **Text Generation**: Writing articles, poems, scripts, and code.
* **Music Generation**: Composing original music in various styles.
* **Video Generation**: Creating short videos or animations.
* **Drug Discovery**: Designing new molecules with desired properties.
* **Product Design**: Generating new product ideas and prototypes.
* **Data Augmentation**: Creating synthetic data to improve the performance of other machine learning models.

#### Limitations and Challenges
* **Data Dependency**: Generative AI models require large amounts of training data.
* **Bias**: Generated data can reflect biases present in the training data.
* **Lack of Control**: It can be difficult to control the specific characteristics of the generated data.
* **Ethical Concerns**: Concerns about misuse, such as creating deepfakes or generating misleading content.
* **Computational Cost**: Training generative models can be computationally expensive.

In summary, generative AI is a powerful technology that allows computers to create new data that resembles the data they were trained on. It has numerous applications across various industries, but it also presents ethical and practical challenges that need to be addressed.

In [99]:
result=get_response("give me a more brief explanation")

In [101]:
def render_markdown(text:str):
    display(Markdown(text))

render_markdown(result)

### Generative AI
Generative AI learns from data to create new, similar data. It's like teaching a computer to imitate and then create its own original content (text, images, etc.). 
#### Uses of Generative AI
It's used for things like:
* Generating images
* Writing text
* Composing music
#### Requirements and Concerns
However, it requires lots of data and raises ethical concerns.

In [105]:
config2={"configurable":{"thread_id":"2"}}

jobs=get_response(agent_prompt_template.format(resume=summary),config=config2)
# industry_trends=agent.invoke({"messages":agent_prompt_template.format(resume=summary)})

In [106]:
display(Markdown(jobs))

### Introduction
I am still unable to reliably extract all the required information to fulfill the request completely. However, here are some potential leads based on the search results. Please note that the information might be incomplete, and it is recommended to verify the details on the linked pages.

### Job Leads

1. **Company Name:** Observe AI
    * **Job Role:** Machine Learning Engineer - NLP
    * **Skills Required:** Machine Learning, Deep Learning, NLP
    * **Location:** Not specified in the summary, check the link for details.
    * **Salary:** Not specified
    * **Application Link:** [https://www.moaijobs.com/job/machine-learning-engineer-nlp-observe-ai-4210](https://www.moaijobs.com/job/machine-learning-engineer-nlp-observe-ai-4210)

2. **Company Name:** IQVIA
    * **Job Role:** AI Engineer
    * **Skills Required:** Python, TensorFlow, PyTorch, Hugging Face (Likely others, see link)
    * **Location:** Bengaluru
    * **Salary:** Not specified
    * **Application Link:** [https://www.linkedin.com/posts/niranjan-m-084a4145_iqvia-now-hiring-ai-engineer-activity-723244855568924442828-avzt](https://www.linkedin.com/posts/niranjan-m-084a4145_iqvia-now-hiring-ai-engineer-activity-723244855568924442828-avzt)

3. **Company Name:** Coditas
    * **Job Role:** Generative AI Engineer - Python
    * **Skills Required:** Azure, Python, TensorFlow, PyTorch, scikit-learn
    * **Location:** Not specified, check the link for details.
    * **Salary:** Not specified
    * **Application Link:** [https://www.linkedin.com/posts/coditas_hiring-now-langchain-rag-activity-7272508141416689664-blDP](https://www.linkedin.com/posts/coditas_hiring-now-langchain-rag-activity-7272508141416689664-blDP)

### Conclusion
I am continuing to improve my ability to extract this information. In the meantime, please use these leads as a starting point for your job search and verify the details on the application pages.

In [107]:
parser.parse(jobs)

OutputParserException: Invalid json output: ### Introduction
I am still unable to reliably extract all the required information to fulfill the request completely. However, here are some potential leads based on the search results. Please note that the information might be incomplete, and it is recommended to verify the details on the linked pages.

### Job Leads

1. **Company Name:** Observe AI
    * **Job Role:** Machine Learning Engineer - NLP
    * **Skills Required:** Machine Learning, Deep Learning, NLP
    * **Location:** Not specified in the summary, check the link for details.
    * **Salary:** Not specified
    * **Application Link:** [https://www.moaijobs.com/job/machine-learning-engineer-nlp-observe-ai-4210](https://www.moaijobs.com/job/machine-learning-engineer-nlp-observe-ai-4210)

2. **Company Name:** IQVIA
    * **Job Role:** AI Engineer
    * **Skills Required:** Python, TensorFlow, PyTorch, Hugging Face (Likely others, see link)
    * **Location:** Bengaluru
    * **Sa

In [108]:
jobs

'### Introduction\nI am still unable to reliably extract all the required information to fulfill the request completely. However, here are some potential leads based on the search results. Please note that the information might be incomplete, and it is recommended to verify the details on the linked pages.\n\n### Job Leads\n\n1. **Company Name:** Observe AI\n    * **Job Role:** Machine Learning Engineer - NLP\n    * **Skills Required:** Machine Learning, Deep Learning, NLP\n    * **Location:** Not specified in the summary, check the link for details.\n    * **Salary:** Not specified\n    * **Application Link:** [https://www.moaijobs.com/job/machine-learning-engineer-nlp-observe-ai-4210](https://www.moaijobs.com/job/machine-learning-engineer-nlp-observe-ai-4210)\n\n2. **Company Name:** IQVIA\n    * **Job Role:** AI Engineer\n    * **Skills Required:** Python, TensorFlow, PyTorch, Hugging Face (Likely others, see link)\n    * **Location:** Bengaluru\n    * **Salary:** Not specified\n    

In [111]:
import requests

url = "https://jsearch.p.rapidapi.com/estimated-salary"

querystring = {"job_title":"nodejs developer","location":"new york","location_type":"ANY","years_of_experience":"ALL"}

headers = {
	"x-rapidapi-key": "6d4aea0084msh01be36a5c4c8990p14b81ajsn75774757b89d",
	"x-rapidapi-host": "jsearch.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'status': 'OK', 'request_id': '38e3b239-81e7-43c2-a906-560762ae043f', 'parameters': {'job_title': 'nodejs developer', 'location': 'new york', 'location_type': 'ANY', 'years_of_experience': None}, 'data': [{'location': 'New York City, NY', 'job_title': 'Nodejs Developer', 'min_salary': 111927.29, 'max_salary': 186051.34, 'median_salary': 143224.42, 'min_base_salary': 81995.46, 'max_base_salary': 130178.59, 'median_base_salary': 103315.31, 'min_additional_pay': 29931.83, 'max_additional_pay': 55872.75, 'median_additional_pay': 39909.11, 'salary_period': 'YEAR', 'salary_currency': 'USD', 'salary_count': 60, 'salaries_updated_at': '2024-06-06T23:59:59.000Z', 'publisher_name': 'Glassdoor', 'publisher_link': 'https://www.glassdoor.com/Salaries/company-salaries.htm?suggestCount=0&suggestChosen=false&sc.keyword=Nodejs%20Developer&locT=C&locId=1132348', 'confidence': 'CONFIDENT'}]}


In [114]:
result=response.json()
result

{'status': 'OK',
 'request_id': '38e3b239-81e7-43c2-a906-560762ae043f',
 'parameters': {'job_title': 'nodejs developer',
  'location': 'new york',
  'location_type': 'ANY',
  'years_of_experience': None},
 'data': [{'location': 'New York City, NY',
   'job_title': 'Nodejs Developer',
   'min_salary': 111927.29,
   'max_salary': 186051.34,
   'median_salary': 143224.42,
   'min_base_salary': 81995.46,
   'max_base_salary': 130178.59,
   'median_base_salary': 103315.31,
   'min_additional_pay': 29931.83,
   'max_additional_pay': 55872.75,
   'median_additional_pay': 39909.11,
   'salary_period': 'YEAR',
   'salary_currency': 'USD',
   'salary_count': 60,
   'salaries_updated_at': '2024-06-06T23:59:59.000Z',
   'publisher_name': 'Glassdoor',
   'publisher_link': 'https://www.glassdoor.com/Salaries/company-salaries.htm?suggestCount=0&suggestChosen=false&sc.keyword=Nodejs%20Developer&locT=C&locId=1132348',
   'confidence': 'CONFIDENT'}]}

In [115]:
import requests

url = "https://upwork-jobs-api2.p.rapidapi.com/active-freelance-1h"

querystring = {"location_filter":"\"United States\"","limit":"10"}

headers = {
	"x-rapidapi-key": "6d4aea0084msh01be36a5c4c8990p14b81ajsn75774757b89d",
	"x-rapidapi-host": "upwork-jobs-api2.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

[]


In [118]:
response.json()

[]

In [119]:
import requests

url = "https://glassdoor-real-time.p.rapidapi.com/companies/interview-details"

querystring = {"interviewId":"19018219"}

headers = {
	"x-rapidapi-key": "6d4aea0084msh01be36a5c4c8990p14b81ajsn75774757b89d",
	"x-rapidapi-host": "glassdoor-real-time.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'data': {'employerInterviewDetails': {'__typename': 'InterviewIG', 'advice': None, 'countHelpful': 0, 'difficulty': 'DIFFICULT', 'employer': {'name': 'ENTRUST Solutions Group', 'squareLogoUrl': 'https://media.glassdoor.com/sql/348371/entrust-solutions-group-squareLogo-1670521079923.png'}, 'employerResponses': [], 'experience': 'POSITIVE', 'featured': False, 'id': 19018219, 'jobTitle': {'text': 'Intern - Hourly'}, 'negotiationDescription': None, 'outcome': 'NO_OFFER', 'processDescription': 'Recruiter reached out to me over LinkedIn and asked if I was interested in an internship, then asked me for my information and set up an interview in their Denver office. A panel of engineers was conducting the interview. It was fairly straightforward, with some technical questions thrown in the mix.', 'reviewDateTime': '2018-02-01T03:57:16.780Z', 'source': 'APPLIED_ONLINE', 'userQuestions': [{'answerCount': 0, 'question': 'What are your three greatest accomplishments?'}, {'answerCount': 0, 'questio

In [120]:
response.json()

{'data': {'employerInterviewDetails': {'__typename': 'InterviewIG',
   'advice': None,
   'countHelpful': 0,
   'difficulty': 'DIFFICULT',
   'employer': {'name': 'ENTRUST Solutions Group',
    'squareLogoUrl': 'https://media.glassdoor.com/sql/348371/entrust-solutions-group-squareLogo-1670521079923.png'},
   'employerResponses': [],
   'experience': 'POSITIVE',
   'featured': False,
   'id': 19018219,
   'jobTitle': {'text': 'Intern - Hourly'},
   'negotiationDescription': None,
   'outcome': 'NO_OFFER',
   'processDescription': 'Recruiter reached out to me over LinkedIn and asked if I was interested in an internship, then asked me for my information and set up an interview in their Denver office. A panel of engineers was conducting the interview. It was fairly straightforward, with some technical questions thrown in the mix.',
   'reviewDateTime': '2018-02-01T03:57:16.780Z',
   'source': 'APPLIED_ONLINE',
   'userQuestions': [{'answerCount': 0,
     'question': 'What are your three gr

In [123]:
import requests

url = "https://glassdoor-real-time.p.rapidapi.com/companies/interview-details"

querystring = {
    "page": "1",
    "location": "United States",  # You can change this to "India", "United Kingdom", etc.
    "title": "Data Scientist",    # Optional: Role/title filter
    "company": "Google"           # Optional: Company name filter
}

headers = {
    "x-rapidapi-key": "6d4aea0084msh01be36a5c4c8990p14b81ajsn75774757b89d",
    "x-rapidapi-host": "glassdoor-real-time.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

if response.status_code == 200:
    print(response.json())
else:
    print("Error:", response.status_code, response.text)


{'data': None, 'errors': {'interviewId': 'interviewId is required'}, 'status': False, 'message': 'Errors'}


In [146]:
import requests

url = "https://jobs-search-api.p.rapidapi.com/getjobs"

payload = {
	"search_term": "data",
	"location": ["mumbai","delhi","kolkata",'bengaluru',"hyderabad","new york city"],
	"results_wanted": 5,
	"site_name": ["indeed", "linkedin", "zip_recruiter", "glassdoor"],
	"distance": 50,
	# "job_type": "fulltime",
	# "is_remote": False,
	"linkedin_fetch_description": False,
	"hours_old": 72
}
headers = {
	"x-rapidapi-key": "6d4aea0084msh01be36a5c4c8990p14b81ajsn75774757b89d",
	"x-rapidapi-host": "jobs-search-api.p.rapidapi.com",
	"Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'message': 'some error ocurred', 'status': False}


In [133]:
response.json()["jobs"]

[{'id': 'li-4200971051',
  'site': 'linkedin',
  'job_url': 'https://www.linkedin.com/jobs/view/4200971051',
  'job_url_direct': '',
  'title': 'UI Developer_Associate_Software Engineering',
  'company': 'Morgan Stanley',
  'location': 'Mumbai, Maharashtra, India',
  'date_posted': '2025-04-04',
  'job_type': '',
  'salary_source': '',
  'interval': '',
  'min_amount': '',
  'max_amount': '',
  'currency': '',
  'is_remote': 'False',
  'job_level': '',
  'job_function': '',
  'listing_type': '',
  'emails': '',
  'description': '',
  'company_industry': '',
  'company_url': 'https://www.linkedin.com/company/morgan-stanley',
  'company_logo': '',
  'company_url_direct': '',
  'company_addresses': '',
  'company_num_employees': '',
  'company_revenue': '',
  'company_description': '',
  'skills': '',
  'experience_range': '',
  'company_rating': '',
  'company_reviews_count': '',
  'vacancy_count': '',
  'work_from_home_type': ''},
 {'id': 'li-4199480133',
  'site': 'linkedin',
  'job_ur

In [139]:
role=[]
job_url=[]
company=[]
location=[]
date_posted=[]
salary=[]

for job in response.json()["jobs"]:
    role.append(job['title'])
    job_url.append(job["job_url"])
    company.append(job["company"])
    location.append(job["location"])
    date_posted.append(job["date_posted"])
    salary.append(f"{job["min_amount"]} - {job["max_amount"]}")

In [141]:
import pandas as pd

df=pd.DataFrame(
    {
        "company":company,
        "role":role,
        "job_url":job_url,
        "location":location,
        "date_posted":date_posted,
        "salary":salary
    }
)

df

,company,role,job_url,location,date_posted,salary
0,Morgan Stanley,UI Developer_Associate_Software Engineering,https://www.linkedin.com/jobs/view/4200971051,"Mumbai, Maharashtra, India",2025-04-04,-
1,Outlier,Web Developer,https://www.linkedin.com/jobs/view/4199480133,,2025-04-04,-
2,Morgan Stanley,Marketing Operations and Digital Associate,https://www.linkedin.com/jobs/view/4199722929,"Mumbai, Maharashtra, India",2025-04-04,-
3,Outlier,Web Developer,https://www.linkedin.com/jobs/view/4199479294,"Thane, Maharashtra, India",2025-04-04,-
4,Amazon,HR Ops Admin,https://www.linkedin.com/jobs/view/4121157377,"Thane, Maharashtra, India",,-


In [ ]:
def search_jobs_rapid_api():
    url = "https://jobs-search-api.p.rapidapi.com/getjobs"

    payload = {
        "search_term": "web",
        "location": "mumbai",
        "results_wanted": 5,
        "site_name": ["indeed", "linkedin", "zip_recruiter", "glassdoor"],
        "distance": 50,
        "job_type": "fulltime",
        "is_remote": False,
        "linkedin_fetch_description": False,
        "hours_old": 72
    }
    headers = {
        "x-rapidapi-key": "6d4aea0084msh01be36a5c4c8990p14b81ajsn75774757b89d",
        "x-rapidapi-host": "jobs-search-api.p.rapidapi.com",
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

In [147]:
import requests

url = "https://indeed12.p.rapidapi.com/company/Ubisoft/jobs"

querystring = {"title":"data-scientist","start":"1"}

headers = {
	"x-rapidapi-key": "6d4aea0084msh01be36a5c4c8990p14b81ajsn75774757b89d",
	"x-rapidapi-host": "indeed12.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'count': 0, 'hits': [], 'indeed_final_url': 'https://www.indeed.com/cmp/Ubisoft/jobs?clearPrefilter=1&start=1'}


In [162]:
jobs=get_response(agent_prompt_template.format(resume=summary),config=config)

In [163]:
display(Markdown(jobs))

### Based on the search results, here are a few potential job opportunities that align with Soham Mukherjee's profile.
Since the previous search did not give specific job postings, I have used a broader search query. Please note that due to the limitations of the search tool, I may not be able to verify the current status of these postings. Always check the application link to confirm the job is still available and to get the most accurate details.

#### Job Opportunities
1. **Company Name:** Pyxis
	* **Job Role:** Machine Learning Engineer
	* **Skills Required:** Computer Vision, Natural Language Processing (NLP), Generative AI, Deep Learning
	* **Location:** Not specified
	* **Salary:** Not specified
	* **Application Link:** [https://pyxis.tech/jobs/machine-learning-engineer/](https://pyxis.tech/jobs/machine-learning-engineer/)
2. **Company Name:** F(x) Data Labs
	* **Job Role:** AI Engineer – LLM Specialist
	* **Skills Required:** Machine Learning, Generative AI, NLP, Computer Vision, Deep Learning & LLM Expertise
	* **Location:** Not specified
	* **Salary:** Not specified
	* **Application Link:** [https://wellfound.com/jobs/3235842-ai-engineer-llm-specialist](https://wellfound.com/jobs/3235842-ai-engineer-llm-specialist)
3. **Company Name:** aijobs.net (Aggregator for LangChain related jobs)
	* **Job Role:** Various - AI Engineer, Machine Learning Engineer, and related roles
	* **Skills Required:** Varies depending on the specific job posting, but generally includes LangChain, AI/ML, Python
	* **Location:** Varies, global opportunities
	* **Salary:** Not specified
	* **Application Link:** [https://aijobs.net/list/langchain-related-jobs/](https://aijobs.net/list/langchain-related-jobs/) (This is a job board, you'll need to navigate to individual postings)

### Note
I am unable to provide 5 specific job postings with complete information at this time due to the limitations of the tool and the nature of the search results. It is recommended to use dedicated job search platforms for a more comprehensive search.

In [164]:
get_response("tell me about neural networks",config=config)

"### Introduction to Neural Networks\nNeural networks are a set of algorithms, modeled loosely after the human brain, that are designed to recognize patterns. They interpret sensory data through a kind of machine perception, labeling or clustering raw input. The patterns they recognize are numerical, contained in vectors, into which all real-world data, be it images, sound, text, or time series, must be translated.\n\n### Key Aspects of Neural Networks\n#### Core Concepts:\n*   **Neurons (Nodes)**: The basic building blocks of a neural network. Each neuron receives input, processes it, and produces an output.\n*   **Connections (Edges/Weights)**: Neurons are connected to each other through connections. Each connection has a weight associated with it, which represents the strength of the connection.\n*   **Layers**: Neurons are organized into layers:\n    *   **Input Layer**: Receives the initial data.\n    *   **Hidden Layers**: Perform intermediate computations. A neural network can h